# Orca: An Agentic Multi-Model Assistant for Stock Analysis & Forecasting

**Google 5-Day AI Agents Intensive – Capstone Project**

---

## Abstract

Orca is an AI stock analysis agent that helps users understand a stock from multiple angles: price history, technical indicators, ML forecasts, and natural-language explanations.

The system is built using the **model + tools + orchestration** pattern from Google's agent architecture:
- **Model**: Multi-LLM reasoning layer with judge-based selection
- **Tools**: Python functions for data fetching, indicators, forecasting, and quantum-inspired risk
- **Orchestration**: Think → Act → Observe cycle

This notebook demonstrates sessions & memory, observability & evaluation, and an AI Canvas workflow visualization.

## 1. Setup & Imports

In [ ]:
# Core libraries
import os
import math
import json
import time
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional

# Data processing
import numpy as np
import pandas as pd

# Market data
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec

# Advanced Math
try:
    from statsmodels.tsa.arima.model import ARIMA
    print("✓ Statsmodels imported for ARIMA")
except ImportError:
    print("! Statsmodels not found, ARIMA will be disabled")

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

## 2. Configuration & Helper Types

In [ ]:
@dataclass
class IndicatorResult:
    """Result from technical indicator computation"""
    df: pd.DataFrame
    metrics: Dict[str, float]

@dataclass
class ForecastResult:
    """Result from ML forecast"""
    direction: str        # "UP" or "DOWN"
    expected_change_pct: float
    confidence: float
    horizon_days: int

@dataclass
class QuantumRiskResult:
    """Result from quantum-inspired risk analysis"""
    quantum_risk: float
    trade_probability: float

@dataclass
class AgentTraceEvent:
    """Single event in agent execution trace"""
    step: int
    component: str    # "tool", "model", "judge"
    name: str
    status: str
    inputs: Dict[str, Any]
    outputs: Dict[str, Any]
    timestamp: float

# Global trace log
agent_trace: List[AgentTraceEvent] = []

def log_event(component: str, name: str, status: str, inputs: Dict[str, Any], outputs: Dict[str, Any]):
    """Log an agent execution event for observability"""
    agent_trace.append(
        AgentTraceEvent(
            step=len(agent_trace) + 1,
            component=component,
            name=name,
            status=status,
            inputs=inputs,
            outputs=outputs,
            timestamp=time.time()
        )
    )
    print(f"[{component.upper()}] {name}: {status}")

print("✓ Data structures defined")

## 3. Agent Tools – Data Fetching & Technical Indicators

In [ ]:
def tool_fetch_price_data(symbol: str, period="1y", interval="1d") -> pd.DataFrame:
    """Fetch OHLCV data for a symbol using yfinance."""
    df = yf.download(symbol, period=period, interval=interval, auto_adjust=True, progress=False)
    df = df.dropna()
    log_event("tool", "fetch_price_data", "ok", {"symbol": symbol}, {"rows": len(df)})
    return df

def tool_compute_indicators(df: pd.DataFrame) -> IndicatorResult:
    """Compute advanced technical indicators from price data."""
    data = df.copy()
    
    # Moving averages
    data["MA20"] = data["Close"].rolling(20).mean()
    data["MA50"] = data["Close"].rolling(50).mean()
    
    # Daily returns & volatility
    data["returns"] = data["Close"].pct_change()
    vol = data["returns"].std() * np.sqrt(252) * 100
    
    # RSI
    delta = data["Close"].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = -delta.clip(upper=0).rolling(14).mean()
    rs = gain / (loss + 1e-9)
    data["RSI"] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = data["Close"].ewm(span=12, adjust=False).mean()
    ema26 = data["Close"].ewm(span=26, adjust=False).mean()
    data["MACD"] = ema12 - ema26
    data["MACD_signal"] = data["MACD"].ewm(span=9, adjust=False).mean()
    
    # Bollinger Bands
    bb_std = data["Close"].rolling(20).std()
    data["BB_Upper"] = data["MA20"] + (bb_std * 2)
    data["BB_Lower"] = data["MA20"] - (bb_std * 2)
    data["BB_Pos"] = (data["Close"] - data["BB_Lower"]) / (data["BB_Upper"] - data["BB_Lower"])
    
    # Williams %R
    highest_high = data["High"].rolling(14).max()
    lowest_low = data["Low"].rolling(14).min()
    data["Williams_%R"] = -100 * (highest_high - data["Close"]) / (highest_high - lowest_low)
    
    # ATR
    tr1 = data["High"] - data["Low"]
    tr2 = abs(data["High"] - data["Close"].shift(1))
    tr3 = abs(data["Low"] - data["Close"].shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    data["ATR"] = tr.rolling(14).mean()
    
    # Performance metrics
    total_return = (data["Close"].iloc[-1] / data["Close"].iloc[0] - 1) * 100
    sharpe = (data["returns"].mean() / (data["returns"].std() + 1e-9)) * np.sqrt(252)
    
    # Max drawdown
    cum_max = data["Close"].cummax()
    drawdown = (data["Close"] / cum_max) - 1
    max_dd = drawdown.min() * 100
    
    metrics = {
        "volatility_pct": float(vol),
        "total_return_pct": float(total_return),
        "sharpe": float(sharpe),
        "max_drawdown_pct": float(max_dd),
        "rsi": float(data["RSI"].iloc[-1]),
        "macd": float(data["MACD"].iloc[-1]),
        "williams_r": float(data["Williams_%R"].iloc[-1]),
        "atr": float(data["ATR"].iloc[-1])
    }
    
    log_event("tool", "compute_indicators", "ok", {}, metrics)
    return IndicatorResult(df=data, metrics=metrics)

print("✓ Data fetching and advanced indicator tools defined")

## 4. Agent Tools – ML Forecast

In [ ]:
def tool_run_forecast(df: pd.DataFrame, horizon_days: int = 30) -> ForecastResult:
    """Run hybrid forecast (ARIMA + Linear Regression)."""
    prices = df["Close"].values
    
    # 1. Linear Regression (Trend)
    X = np.arange(len(prices))
    coeffs = np.polyfit(X, prices, 1)
    m, b = coeffs
    
    future_X = np.arange(len(prices), len(prices) + horizon_days)
    lin_pred = m * future_X + b
    
    # 2. ARIMA (Short-term dynamics)
    try:
        # Simple ARIMA(5,1,0) on recent data
        model = ARIMA(prices[-100:], order=(5,1,0))
        model_fit = model.fit()
        arima_forecast = model_fit.forecast(steps=horizon_days)
        
        # Combine: Average of both for robustness
        final_pred = (lin_pred + arima_forecast) / 2
        method = "Hybrid (ARIMA + Linear)"
    except:
        final_pred = lin_pred
        method = "Linear Regression"
    
    expected_change_pct = (final_pred[-1] - prices[-1]) / prices[-1] * 100
    
    # Confidence based on R^2 of linear fit
    pred_past = m * X + b
    ss_res = np.sum((prices - pred_past) ** 2)
    ss_tot = np.sum((prices - prices.mean()) ** 2) + 1e-9
    r2 = 1 - ss_res / ss_tot
    confidence = float(np.clip(r2 * 100, 50, 95))
    
    direction = "UP" if expected_change_pct >= 0 else "DOWN"
    
    out = {
        "direction": direction,
        "expected_change_pct": expected_change_pct,
        "confidence": confidence,
        "horizon_days": horizon_days,
    }
    
    log_event("tool", "run_forecast", "ok", 
              {"method": method, "horizon": horizon_days}, out)
    return ForecastResult(**out)

print("✓ Hybrid ML forecast tool defined")

## 5. Agent Tools – Quantum-Inspired Risk

In [ ]:
def tool_quantum_risk(indicators: IndicatorResult) -> QuantumRiskResult:
    """Compute quantum-inspired risk metrics.
    
    We scale volatility by 1.2 to model extra uncertainty, then convert it 
    into a probability with a 0.7 damping factor so values stay realistic.
    
    Args:
        indicators: IndicatorResult with computed metrics
    
    Returns:
        QuantumRiskResult with risk score and trade probability
    """
    vol = indicators.metrics["volatility_pct"]
    quantum_risk = vol * 1.2            # amplify uncertainty slightly
    trade_prob = max(0.0, 100.0 - 0.7 * quantum_risk)
    
    out = {
        "quantum_risk": float(quantum_risk),
        "trade_probability": float(trade_prob),
    }
    
    log_event("tool", "quantum_risk", "ok", {}, out)
    return QuantumRiskResult(**out)

print("✓ Quantum risk tool defined")

## 6. Multi-LLM Orchestration & Judge

In [ ]:
def call_expert_model(model_name: str, analysis_payload: Dict[str, Any]) -> str:
    """Simulate LLM expert model call.
    
    In production, this would call Gemini/GPT/Claude with a prompt that includes:
    - Summary of indicators
    - Forecast
    - Quantum risk
    - User question
    
    Args:
        model_name: Name of the expert model
        analysis_payload: Complete analysis context
    
    Returns:
        Generated text response
    """
    # For demo, return a synthetic answer based on metrics
    direction = analysis_payload["forecast"]["direction"]
    change_pct = analysis_payload["forecast"]["expected_change_pct"]
    quantum_risk = analysis_payload["quantum"]["quantum_risk"]
    
    response = (
        f"As {model_name}, I see a {direction} trend with ~{change_pct:.2f}% change "
        f"and quantum risk {quantum_risk:.1f}. This looks like a "
        f"{'cautious buy' if direction=='UP' else 'cautious sell'}."
    )
    
    log_event("model", f"expert_{model_name}", "ok", 
              {"model": model_name}, {"response_len": len(response)})
    
    return response

def judge_answers(judge_model: str, answers: Dict[str, str], context: Dict[str, Any]) -> Dict[str, Any]:
    """LM-as-a-Judge pattern: score and rank expert answers.
    
    In production, this would send all answers to a judge LLM (e.g., DeepSeek-R1)
    to evaluate quality, accuracy, and relevance.
    
    Args:
        judge_model: Name of the judge model
        answers: Dict mapping model names to their responses
        context: Analysis context for evaluation
    
    Returns:
        Dict with scores and best answer
    """
    scored = []
    for model, text in answers.items():
        # Heuristic scoring for demo (in production, use actual LLM judge)
        score = 7.0 + np.random.uniform(-1, 2)  # Simulate variation
        scored.append({"model": model, "answer": text, "score": score})
    
    best = max(scored, key=lambda x: x["score"])
    
    log_event("model", "judge_answers", "ok", 
              {"n_models": len(answers)}, {"best_model": best["model"]})
    
    return {"scores": scored, "best": best}

print("✓ Multi-LLM orchestration defined")

## 7. Agent Orchestration Loop

In [ ]:
class OrcaAgent:
    """Main agent orchestrator implementing the Think → Act → Observe cycle."""
    
    def __init__(self, expert_models: List[str], judge_model: str):
        self.expert_models = expert_models
        self.judge_model = judge_model
        self.sessions = {}  # session_id -> history/memory (simple dict)
    
    def analyze_stock(self, session_id: str, symbol: str, question: str, horizon_days=30):
        """Execute full agent workflow for stock analysis.
        
        Args:
            session_id: User session identifier
            symbol: Stock ticker symbol
            question: User's question about the stock
            horizon_days: Forecast horizon in days
        
        Returns:
            Tuple of (analysis_payload, judge_result)
        """
        print(f"\n{'='*60}")
        print(f"AGENT ANALYZING: {symbol}")
        print(f"QUESTION: {question}")
        print(f"{'='*60}\n")
        
        # 1) THINK: Fetch & compute (tool calls)
        df = tool_fetch_price_data(symbol)
        indicators = tool_compute_indicators(df)
        forecast = tool_run_forecast(indicators.df, horizon_days=horizon_days)
        quantum = tool_quantum_risk(indicators)
        
        # Combine numerical analysis into a single payload
        analysis_payload = {
            "symbol": symbol,
            "indicators": indicators.metrics,
            "forecast": asdict(forecast),
            "quantum": asdict(quantum),
            "question": question,
        }
        
        # 2) ACT: Multi-LLM panel generates responses
        answers = {}
        for model_name in self.expert_models:
            text = call_expert_model(model_name, analysis_payload)
            answers[model_name] = text
        
        # 3) OBSERVE: Judge evaluates and selects best answer
        judge_result = judge_answers(self.judge_model, answers, analysis_payload)
        
        # 4) MEMORY: Update session history
        history = self.sessions.setdefault(session_id, [])
        history.append({
            "symbol": symbol,
            "question": question,
            "analysis": analysis_payload,
            "judge_result": judge_result,
        })
        
        print(f"\n✓ Analysis complete. Best model: {judge_result['best']['model']}")
        
        return analysis_payload, judge_result

print("✓ Agent orchestration class defined")

## 8. Demo Run – Analyze AAPL

In [ ]:
# Initialize agent with expert models
agent = OrcaAgent(
    expert_models=["Expert-A", "Expert-B", "Expert-C"],
    judge_model="Judge-1"
)

# Run analysis
analysis, judge_result = agent.analyze_stock(
    session_id="demo_user_1",
    symbol="AAPL",
    question="Is this a good short-term buy for the next month?",
    horizon_days=30
)

# Display results
print("\n" + "="*60)
print("ANALYSIS RESULTS")
print("="*60)
print(f"\nSymbol: {analysis['symbol']}")
print(f"\nIndicators:")
for key, value in analysis['indicators'].items():
    print(f"  {key}: {value:.2f}")
print(f"\nForecast:")
for key, value in analysis['forecast'].items():
    print(f"  {key}: {value}")
print(f"\nQuantum Risk:")
for key, value in analysis['quantum'].items():
    print(f"  {key}: {value:.2f}")
print(f"\nBest Answer (from {judge_result['best']['model']}):")
print(f"  {judge_result['best']['answer']}")
print(f"  Score: {judge_result['best']['score']:.2f}/10")

## 9. Visualization – Price Chart with Indicators

In [ ]:
# Fetch data for visualization
df = tool_fetch_price_data("AAPL")
ind_res = tool_compute_indicators(df)

# Create figure
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), 
                                gridspec_kw={'height_ratios': [3, 1]})

# Price chart with moving averages
ax1.plot(ind_res.df.index, ind_res.df["Close"], label="Close", linewidth=2, color='#06b6d4')
ax1.plot(ind_res.df.index, ind_res.df["MA20"], label="MA20", linewidth=1.5, 
         color='#f59e0b', linestyle='--')
ax1.plot(ind_res.df.index, ind_res.df["MA50"], label="MA50", linewidth=1.5, 
         color='#8b5cf6', linestyle='--')
ax1.set_title("AAPL Price with Moving Averages", fontsize=16, fontweight='bold')
ax1.set_ylabel("Price ($)", fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# RSI chart
ax2.plot(ind_res.df.index, ind_res.df["RSI"], label="RSI", linewidth=2, color='#10b981')
ax2.axhline(y=70, color='#ef4444', linestyle='--', linewidth=1, alpha=0.7, label='Overbought')
ax2.axhline(y=30, color='#10b981', linestyle='--', linewidth=1, alpha=0.7, label='Oversold')
ax2.set_title("Relative Strength Index (RSI)", fontsize=14)
ax2.set_ylabel("RSI", fontsize=12)
ax2.set_xlabel("Date", fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

print("✓ Visualization complete")

## 10. Observability – Agent Trace Table

This trace is our "glass box" view of the agent's trajectory, matching the Agent Quality whitepaper's idea of logging, tracing, and metrics.

Each row represents a single step in the agent's execution, showing:
- **Component**: Whether it's a tool call, model call, or judge decision
- **Name**: Specific function or model invoked
- **Status**: Success/failure status
- **Timestamp**: When the event occurred

This enables debugging, performance analysis, and understanding of the agent's decision-making process.

In [ ]:
# Convert trace to DataFrame
trace_df = pd.DataFrame([asdict(e) for e in agent_trace])
trace_df = trace_df.sort_values("step")

# Display trace table
print("\n" + "="*80)
print("AGENT EXECUTION TRACE")
print("="*80)
print(trace_df[["step", "component", "name", "status", "timestamp"]].to_string(index=False))

# Summary statistics
print(f"\nTotal steps: {len(trace_df)}")
print(f"Tool calls: {len(trace_df[trace_df['component'] == 'tool'])}")
print(f"Model calls: {len(trace_df[trace_df['component'] == 'model'])}")
print(f"Total execution time: {trace_df['timestamp'].max() - trace_df['timestamp'].min():.2f}s")

## 11. Evaluation – Model Scoring

In [ ]:
# Extract and display model scores
scores_df = pd.DataFrame(judge_result['scores'])
scores_df = scores_df.sort_values('score', ascending=False)

print("\n" + "="*60)
print("MODEL EVALUATION SCORES")
print("="*60)
print(scores_df.to_string(index=False))

# Visualize scores
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(scores_df['model'], scores_df['score'], color='#06b6d4')

# Highlight best model
best_idx = scores_df['score'].idxmax()
bars[best_idx].set_color('#10b981')

ax.set_xlabel('Score (0-10)', fontsize=12)
ax.set_title('LLM Expert Model Scores (Judge Evaluation)', fontsize=14, fontweight='bold')
ax.set_xlim(0, 10)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("✓ Evaluation complete")

## 12. AI Canvas – Workflow Visualization

The agent workflow can be visualized as a directed graph:

```
User Question
      |
      v
[Agent Orchestrator]
      |
      +--> [tool_fetch_price_data]
      |
      +--> [tool_compute_indicators]
      |
      +--> [tool_run_forecast]
      |
      +--> [tool_quantum_risk]
      |
      +--> [Expert LLM A]
      +--> [Expert LLM B]
      +--> [Expert LLM C]
      |
      +--> [Judge LLM]
      |
      v
Final Answer + Report
```

In the full web application, this is visualized as an interactive drag-and-drop canvas with nodes and connections, similar to n8n or LangFlow. Users can:
- See real-time execution status
- Inspect inputs/outputs at each node
- Modify the workflow by adding/removing models
- Export the workflow configuration

## 13. Sessions & Memory

In [ ]:
# Demonstrate session memory
print("\n" + "="*60)
print("SESSION MEMORY")
print("="*60)

session_history = agent.sessions.get("demo_user_1", [])
print(f"\nSession ID: demo_user_1")
print(f"Total queries: {len(session_history)}")

for i, entry in enumerate(session_history, 1):
    print(f"\nQuery {i}:")
    print(f"  Symbol: {entry['symbol']}")
    print(f"  Question: {entry['question']}")
    print(f"  Best Model: {entry['judge_result']['best']['model']}")
    print(f"  Score: {entry['judge_result']['best']['score']:.2f}")

# Simulate user preferences (memory)
user_preferences = {
    "demo_user_1": {
        "favorite_tickers": ["AAPL", "MSFT", "GOOGL"],
        "risk_tolerance": "moderate",
        "preferred_horizon": 30,
        "last_active": time.time()
    }
}

print(f"\nUser Preferences:")
for key, value in user_preferences["demo_user_1"].items():
    print(f"  {key}: {value}")

## 14. Portfolio Analysis (New Feature)

We have added a new capability to analyze user-uploaded portfolios (PDFs, Images) using Gemini 2.0 Flash.

The workflow is:
1. User uploads portfolio document.
2. System extracts text/images.
3. Gemini analyzes holdings, risk, and market alignment using **Persona-Based Context Prompting**.
4. Returns an **Executive Summary** with actionable advice.

This feature is available in the web UI under the "Portfolio Analysis" panel.

## 15. Conclusion & Next Steps

### What We've Demonstrated

This notebook showcases a complete AI agent architecture following the Google 5-Day Agents Intensive principles:

1. **Agent Architecture** (Day 1): Think → Act → Observe cycle with clear separation of concerns
2. **Agent Tools** (Day 2): Well-defined, documented tools with structured inputs/outputs
3. **Sessions & Memory** (Day 3): Per-user context and preference storage
4. **Observability & Evaluation** (Day 4): Complete execution traces and LM-as-a-Judge scoring
5. **Production Readiness** (Day 5): Modular design ready for API deployment

### Key Takeaways

- **Multi-LLM orchestration** reduces hallucination and improves reliability
- **Quantum-inspired signals** add novel risk assessment beyond classical indicators
- **Transparent workflow** enables debugging and trust-building
- **Agentic pattern** provides flexibility and extensibility

### Future Enhancements

- **Real LLM Integration**: Connect to Gemini, GPT-4, or Claude APIs
- **Advanced ML**: Replace linear regression with LSTM or Transformer models
- **Real-time Data**: Stream live market data for intraday trading
- **Portfolio Analysis**: Extend from single-stock to multi-asset portfolios
- **Backtesting**: Validate forecast accuracy over historical periods
- **Web Deployment**: Deploy as Flask/FastAPI service with React frontend

### Resources

- **Full Web Application**: See `app.py` for Flask implementation with AI Canvas
- **Documentation**: See `README.md` for complete project overview
- **Agent Whitepapers**: Google AI Agents Intensive course materials

---

**Thank you for reviewing Orca!**

This project demonstrates how traditional quantitative finance, modern ML, and LLM reasoning can be combined into a transparent, inspectable AI agent suitable for educational and retail-investor use.

## 16. User Guide: How to Use Orca

This section provides a step-by-step guide on how to run and interact with the Orca application.

### Step 1: Installation & Setup
1. **Clone the Repository**:
   ```bash
   git clone https://github.com/Sansyuh06/stocks.git
   cd stocks
   ```
2. **Install Dependencies**:
   ```bash
   pip install -r requirements.txt
   ```
3. **Install Ollama Models** (for local AI):
   ```bash
   ollama pull mistral:7b
   ollama pull llama3.1:8b
   ollama pull phi3:mini
   ollama pull deepseek-r1:7b
   ```

### Step 2: Running the Application
1. Start the Flask server:
   ```bash
   python app.py
   ```
2. Open your web browser and navigate to:
   `http://localhost:5000`

### Step 3: Using the Dashboard
1. **Select a Stock**: Use the dropdown menu or type a ticker symbol (e.g., `NVDA`).
2. **Analyze**: Click the **"Analyze Stock"** button. The agent will:
   - Fetch historical data.
   - Calculate technical indicators (RSI, MACD, etc.).
   - Run the ML forecast.
3. **View Results**: Explore the interactive charts, risk metrics, and trading signals.
4. **Get AI Consensus**: Select your preferred AI models (e.g., Mistral + Llama 3) and click **"Get AI Consensus"** to see a multi-perspective report.

## 17. Component Deep Dive

Understanding the core building blocks of the Orca architecture.

### 1. The Orchestrator (ADK Agent)
- **Role**: The "brain" of the system.
- **Function**: It follows a **Think-Act-Observe** loop. It receives a user query, decides which tools to use, executes them, and synthesizes the results.
- **Code**: Implemented in the `StockAnalyzer` class in `app.py`.

### 2. The Tools
Specialized Python functions that the agent can call:
- **`fetch_data`**: Retrieves OHLCV market data from Yahoo Finance or Alpha Vantage.
- **`calculate_advanced_indicators`**: Computes mathematical metrics like:
  - **RSI**: Momentum oscillator.
  - **Bollinger Bands**: Volatility measurement.
  - **Williams %R**: Overbought/oversold levels.
  - **ATR**: Market volatility.
- **`forecast_short_term_price`**: A hybrid ML model combining **ARIMA** (statistical) and **Linear Regression** (trend) to predict future price direction.

### 3. The Panel of Experts (Models)
- **Role**: Diverse analytical perspectives.
- **Implementation**: We use multiple small, specialized LLMs (via Ollama) instead of one giant model.
  - **Mistral**: Good at general reasoning.
  - **Phi-3**: Excellent for concise, data-driven summaries.
  - **Llama 3**: Strong at structured, deep analysis.

### 4. The Judge (Evaluator)
- **Role**: Quality control.
- **Model**: **DeepSeek-R1**.
- **Function**: It reads the analysis from all experts and scores them (1-10) based on accuracy, depth, and actionability. Only the best response is shown to the user.